In [1]:
import numpy as np
from utils import *
import pandas as pd

In [2]:
data = pd.read_table("data/trec_crowd.txt",header=None)

In [3]:
truth = pd.read_table("data/trec_truth.txt", header=None)


In [4]:
def transform_data(data):
    X = np.array(data)
    rows, _ = X.shape
    n,m,k = np.max(np.array(data), axis=0)
    labels = np.zeros((m,n),dtype=np.int64)-1
    for r in range(rows):
        labels[X[r,1]-1,X[r,0]-1] = X[r,2]-1
    return labels


In [5]:
n,m,k = np.max(np.array(data),axis=0)
print(n,m,k)

19033 762 2


In [6]:
labels = transform_data(data)


In [7]:
labels


array([[ 0,  0,  0, ..., -1, -1, -1],
       [ 1,  1,  0, ..., -1, -1, -1],
       [ 1,  1,  1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], dtype=int64)

In [8]:
groups = np.array([i % 3 for i in range(m)])

In [9]:
np.random.seed(0)
groups = np.random.randint(3, size=m)

In [10]:
C_estimated = get_confusion_matrix(k, labels, groups=groups,sym=False)


In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
i = 50
truth_new = truth[truth[0].isin(data[data[1] == i+1][0])]
pred_new = data[data[1] == i+1][data[data[1] == i+1][0].isin(truth_new[0])]


In [13]:
truth_new

,0,1
19,147,2
282,2280,2
471,3847,1


In [14]:
pred_new

,0,1,2
723,147,51,2
10926,2280,51,2
18234,3847,51,1


In [218]:
C = confusion_matrix(pred_new[2],truth_new[1])

In [219]:
np.round(C / np.sum(C, axis=0)[np.newaxis, :],5)


array([[0.88235, 0.3913 ],
       [0.11765, 0.6087 ]])

In [220]:
np.round(C_estimated[i],5)

array([[0.95657, 0.34086],
       [0.04343, 0.65914]])

In [210]:
m, n = labels.shape


In [15]:
Zg = get_Zg(k, labels, groups)


In [16]:
Zg

array([[[0.003861  , 0.003861  ],
        [0.003861  , 0.003861  ],
        [0.003861  , 0.003861  ],
        ...,
        [0.        , 0.003861  ],
        [0.        , 0.003861  ],
        [0.003861  , 0.003861  ]],

       [[0.        , 0.01176471],
        [0.        , 0.01176471],
        [0.00392157, 0.00784314],
        ...,
        [0.00392157, 0.        ],
        [0.00392157, 0.        ],
        [0.00392157, 0.00392157]],

       [[0.        , 0.        ],
        [0.        , 0.        ],
        [0.        , 0.        ],
        ...,
        [0.01209677, 0.        ],
        [0.00806452, 0.        ],
        [0.00403226, 0.        ]]])

In [17]:
Z = np.zeros((m,n,k))

In [18]:
for i in range(len(data)):
    Z[data.iloc[i, 1]-1, data.iloc[i, 0]-1, data.iloc[i, 2]-1] = 1


In [19]:
Zg = np.zeros((3,n,k))
for g in range(3):
    Zg[g,:,:] = np.mean(Z[groups==g,:,:],axis=0)

In [20]:
np.all(Zg == get_Zg(k, labels, groups))

True

In [21]:
M2s, M3s = get_M(Zg)


In [22]:
a,b,c = 1,2,0
Za = (Zg[c, :, :].T@Zg[b, :, :]/n)@np.linalg.inv(Zg[a, :, :].T @
                                                 Zg[b, :, :]/n)@Zg[a, :, :].T
Zb = (Zg[c, :, :].T@Zg[a, :, :]/n)@np.linalg.inv(Zg[b, :, :].T @
                                                 Zg[a, :, :]/n)@Zg[b, :, :].T

In [23]:
M3 = np.einsum("ji,ki,li->jkl", Za, Zb, Zg[c, :, :].T)/n
M2 = Za@Zb.T/n


In [24]:
u, s, vh = np.linalg.svd(M2, full_matrices=False)


In [25]:
get_whiten(M2, sym=True)


array([[  69.41578696,  700.08859238],
       [ 226.54734822, -214.51233469]])

In [26]:
Q = u@np.diag(s**-0.5)

In [30]:
Q.shape

(5, 5)

In [31]:
M3_whiten = whiten_tensor(M3, Q)

In [46]:
%%timeit
M3_whiten = np.zeros((k,k,k))
for a in range(k):
    for b in range(k):
        for c in range(k):
            for d in range(k):
                for e in range(k):
                    for f in range(k):
                        M3_whiten[a, b, c] += M3[d, e, f] * \
                            Q[d, a]*Q[e, b]*Q[f, c]


68.1 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
%%timeit
M3_whiten = np.einsum("def,da,eb,fc->abc", M3, Q, Q, Q)

5.59 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [336]:
values, vectors = robust_tensor_power(M3_whiten, 20, 100)


In [292]:
np.allclose(M3_new,M3_whiten)

True

In [268]:
M3_2 = np.zeros((k,k,k))

In [54]:
%%timeit
M3_2 = np.zeros((k, k, k))
for i1 in range(k):
    for i2 in range(k):
        for i3 in range(k):
            M3_2[i1, i2, i3] = np.sum(Za[i1,:]*Zb[i2,:]*Zg[c,:,i3])/n

220 µs ± 8.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [53]:
%%timeit
M3 = np.einsum("ji,ki,li->jkl", Za, Zb, Zg[c, :, :].T)/n


617 µs ± 28.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
np.allclose(M3,M3_2)

True

In [39]:
def robust_tensor_power(T, L=20, N=100, sym=True):
    k, _, _ = T.shape
    if sym:
        T = sym_tensor(T)
    values = np.zeros(k)
    vectors = np.zeros((k, k))
    for h in range(k):
        theta_tau = []
        for tau in range(L):
            theta = np.random.randn(k)
            theta /= np.linalg.norm(theta)
            for t in range(N):
                theta = np.einsum("aef,e,f->a", T, theta, theta)
                theta /= np.linalg.norm(theta)
            theta_tau.append(theta)

        lam_best = float("-Inf")
        theta_best = np.array([])
        for theta in theta_tau:
            lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)
            if lam > lam_best:
                lam_best = lam
                theta_best = theta

        theta = theta_best
        for t in range(N):
            theta = np.einsum("aef,e,f->a", T, theta, theta)
            theta /= np.linalg.norm(theta)
        lam = np.einsum("efg,e,f,g->", T, theta, theta, theta)

        values[h] = lam
        vectors[:, h] = theta
        T = T-lam*np.einsum("a,b,c->abc", theta, theta, theta)
    return values, vectors

In [133]:
lam, vec = robust_tensor_power(M3_whiten)

In [132]:
T = sym_tensor(M3_whiten)


In [121]:
lam, vec = robust_tensor_power(T, L = 20,N=100, sym=False)


In [122]:
temp = np.zeros((k, k, k))
for i in range(k):
    temp += lam[i]*np.einsum("a,b,c->abc", vec[:, i], vec[:, i], vec[:, i])

In [123]:
np.mean(np.abs(temp-T))

0.03568052611212279

In [134]:
mu = np.linalg.inv(Q.T)@vec@np.diag(lam)


In [165]:
def get_confusion_matrix(k, labels, groups=None, sym=True,cutoff=1e-7, L=20, N=100, seed=0):
    m, n = labels.shape
    if groups is None:
        np.random.seed(seed)
        groups = np.random.randint(3, size=m)
    Zg = get_Zg(k, labels, groups)
    M2s, M3s = get_M(Zg)
    Cc = np.zeros((3, k, k))
    W = np.zeros((3, k, k))
    for g, (M2, M3) in enumerate(zip(M2s, M3s)):
        Q = get_whiten(M2, sym)
        M3_whiten = whiten_tensor(M3, Q)
        values, vectors = robust_tensor_power(M3_whiten, L, N, sym)
        w = values**-2
        mu = np.linalg.inv(Q.T)@vectors@np.diag(values)
        best = np.argmax(mu, axis=0)
        for h in range(k):
            Cc[g, :, best[h]] = mu[:, h]
            W[g, best[h], best[h]] = w[h]
    W = np.mean(W, axis=0)
    C = np.zeros((m, k, k))
    for i in range(m):
        Ca = (np.sum(Cc, axis=0)-Cc[groups[i], :, :])/2
        Za = (np.sum(Zg, axis=0)-Zg[groups[i], :, :])/2
        E = np.zeros((k, k))
        for j in range(n):
            if labels[i,j]!=-1:
                E[labels[i, j], :] += Za[j, :]
        E /= n
        Ci = E@np.linalg.inv(W@Ca.T)
        if cutoff:
            Ci[Ci < cutoff] = cutoff
        colsums = np.sum(Ci, axis=0)
        Ci /= colsums[np.newaxis, :]
        C[i, :, :] = Ci
    return C


In [166]:
C_estimated = get_confusion_matrix(k, labels, groups=groups, sym=False)


In [136]:
np.argmax(mu,axis=1)

array([0, 2, 1, 4, 3], dtype=int64)

In [138]:
mu0 = lam[0]*np.linalg.inv(Q.T)@vec[:,0]


In [161]:
E = -np.eye(5)

In [163]:
E[E<0]=0

In [164]:
E

array([[ 0., -0., -0., -0., -0.],
       [-0.,  0., -0., -0., -0.],
       [-0., -0.,  0., -0., -0.],
       [-0., -0., -0.,  0., -0.],
       [-0., -0., -0., -0.,  0.]])

In [139]:
np.argmax(mu0)

0

In [82]:
robust_tensor_power(temp,sym=False)


(array([2.47221395, 1.9597989 , 1.68658776, 1.62258932, 1.5157169 ]),
 array([[-0.20816801, -0.45904245, -0.28248499, -0.78507705, -0.44330809],
        [ 0.34572185,  0.49387216,  0.52550452, -0.60544145,  0.20886257],
        [ 0.44722585, -0.25200777,  0.59235523,  0.1237719 , -0.73952102],
        [-0.78471769,  0.16964156,  0.39568989,  0.03355718, -0.24202772],
        [ 0.14611525,  0.67311525, -0.36958358, -0.02547738, -0.39291933]]))

In [83]:
robust_tensor_power(M3_whiten,sym=False)


(array([2.46460466, 1.95134792, 1.69621225, 1.61947364, 1.52595186]),
 array([[-0.19890427, -0.44754347, -0.27192651, -0.7790689 , -0.44857849],
        [ 0.33327075,  0.49026603,  0.52012714, -0.61032911,  0.20734864],
        [ 0.44301318, -0.23003884,  0.60929923,  0.1369673 , -0.7439836 ],
        [-0.79524795,  0.1743316 ,  0.39476733,  0.03790537, -0.24151606],
        [ 0.14383224,  0.68995267, -0.35838099, -0.01879279, -0.37939724]]))